In [1]:
import sys
sys.path.append("../mypkg")

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns

from easydict import EasyDict as edict
from tqdm import trange

plt.style.use('ggplot')
plt.rcParams["savefig.bbox"] = "tight"

In [3]:
from splines import obt_bsp_basis_Rfn
from models.base_model import BaseModel
from models.logistic_model import LogisticModel
from models.linear_model import LinearModel
from utils.matrix import col_vec2mat_fn, col_vec_fn, vecidx2matidx

In [4]:
torch.set_default_tensor_type(torch.DoubleTensor)

### Parameters

In [5]:
paras = edict()
paras.npts = 20 # num of pts to evaluate X(s)
paras.d = 10 # num of ROIs
paras.q = 5 # num of other covariates

paras.bsp = edict()
paras.bsp.aknots_raw = np.linspace(0, 1, 20)
paras.bsp.iknots = paras.bsp.aknots_raw[1:-1]
paras.bsp.bknots = np.array([0, 1])
paras.bsp.ord = 4
paras.bsp.x = np.linspace(0, 1, paras.npts+1)[:-1]
paras.bsp.basis_mat = obt_bsp_basis_Rfn(paras.bsp.x, 
                                        paras.bsp.iknots, 
                                        paras.bsp.bknots, 
                                        paras.bsp.ord)
paras.bsp.N = paras.bsp.basis_mat.shape[1] # num of basis for bsp

In [6]:
M = 30
X = torch.randn(M, paras.d, paras.npts) # M x d x npts
Z = torch.randn(M, paras.q) # M x q
Y = torch.randn(M)
#Y = (Y >0.5).float()

alp = torch.randn(paras.q)
Gam = torch.randn(paras.bsp.N, paras.d) # N x d
basis_mat = torch.DoubleTensor(paras.bsp.basis_mat) # npts x N


In [7]:
model = LinearModel(Y, Z, X, basis_mat)

### Check first dev

In [8]:
dlt = 1e-8 

num_alp = torch.zeros_like(alp)
for ix in range(len(alp)):
    alp_c = alp.clone()
    alp_c[ix] = alp[ix] + dlt

    v1 = model.log_lik(alp, Gam)
    v2 = model.log_lik(alp_c, Gam)
    num_alp[ix] = (v2-v1)/dlt

num_gam = torch.zeros_like(Gam)
for ix in range(Gam.shape[0]):
    for iy in range(Gam.shape[1]):
        Gam_c = Gam.clone()
        Gam_c[ix, iy] = Gam[ix, iy] + dlt*np.sqrt(paras.bsp.N)
        
        v1 = model.log_lik(alp, Gam)
        v2 = model.log_lik(alp, Gam_c)
        num_gam[ix, iy] = (v2-v1)/dlt

In [9]:
grad1 = model.log_lik_der1(alp, Gam)
num_grad = torch.cat([num_alp, col_vec_fn(num_gam)])

In [10]:
diff = grad1 - num_grad
torch.norm(diff)/torch.norm(grad1)

tensor(2.3368e-07)

### Check second dev

In [11]:
dlt = 1e-6

num_the_len = paras.q + paras.d*paras.bsp.N
num_grad2 = torch.zeros(num_the_len, num_the_len)

vraws = model.log_lik_der1(alp, Gam)
for ix in trange(num_the_len):
    alpx = alp.clone()
    Gamx = Gam.clone()
        
    # alp part
    if ix <= (paras.q-1):
        dltx = dlt
        alpx[ix] = alp[ix] + dltx
    else:
        dltx = dlt*np.sqrt(paras.bsp.N)
        loc1, loc2 = vecidx2matidx(ix-paras.q, nrow=paras.bsp.N)
        Gamx[loc1, loc2] = Gam[loc1, loc2] + dltx
            
    v1s = model.log_lik_der1(alpx, Gamx)
    der2_vs = (v1s-vraws)/dlt
        
    #num_grad2[ix, :] = der2_vs
    num_grad2[ix, ix:] = der2_vs[ix:]
num_grad2 = num_grad2 + num_grad2.T - torch.diag(torch.diag(num_grad2))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:00<00:00, 5650.77it/s]


In [12]:
grad2 = model.log_lik_der2(alp, Gam)
diff = grad2 - num_grad2
torch.norm(diff)/torch.norm(grad2)

tensor(1.4394e-09)

In [13]:
mat = (diff.abs()/grad2.abs())
mat.max()

tensor(0.0211)

In [14]:
grad2.abs().min()

tensor(2.1589e-13)

In [15]:
grad2[mat==mat.max()]

tensor([-2.1589e-13, -2.1589e-13])

In [16]:
diff[mat==mat.max()]

tensor([4.5466e-15, 4.5466e-15])

### Check linear form of linear term

linear term = P2\trans \theta

In [53]:
from hdf_utils.likelihood import obt_lin_tm

In [54]:
M = 100
X = torch.randn(M, paras.d, paras.npts) # M x d x npts
Z = torch.randn(M, paras.q) # M x q
Y = torch.randn(M)
#Y = (Y >0.5).float()

alp = torch.randn(paras.q)
Gam = torch.randn(paras.bsp.N, paras.d) # N x d
basis_mat = torch.DoubleTensor(paras.bsp.basis_mat) # npts x N


In [55]:
Os1 = obt_lin_tm(Z, X, alp, Gam, basis_mat)

In [56]:
basis_mat_trans = basis_mat.unsqueeze(0).unsqueeze(-1) # 1 x ncpts x N x 1
X_trans = X.permute((0, 2, 1)).unsqueeze(2) # M x ncpts x 1 x d
        
# derivative of linear term w.r.t (alp, Gam)
vec_part2_raw = basis_mat_trans*X_trans
vec_part2_raw = vec_part2_raw.permute((0, 1, 3, 2)).flatten(2)
#vec_part2 = vec_part2_raw*ws.unsqueeze(0).unsqueeze(-1)
vec_part2 = vec_part2_raw.mean(axis=1)*np.sqrt(basis_mat.shape[1])
tm2 = torch.concat([Z, vec_part2], axis=1) #M x (q+dxN)

theta = torch.cat([alp, col_vec_fn(Gam)/np.sqrt(paras.bsp.N)])
Os2 = tm2 @ theta

In [57]:
Os1 - Os2

tensor([ 8.8818e-16,  6.1062e-16,  8.8818e-16,  1.1102e-16, -8.8818e-16,
         0.0000e+00,  0.0000e+00,  2.2204e-16,  0.0000e+00,  0.0000e+00,
        -6.6613e-16,  8.8818e-16, -1.1102e-16, -5.5511e-17, -8.8818e-16,
         9.4369e-16,  4.4409e-16, -1.1102e-16,  6.6613e-16, -8.8818e-16,
        -8.8818e-16,  0.0000e+00, -1.1102e-15,  0.0000e+00,  0.0000e+00,
        -5.5511e-16,  2.2204e-16, -2.2204e-16,  0.0000e+00,  0.0000e+00,
         8.8818e-16, -4.4409e-16,  0.0000e+00,  3.3307e-16, -1.1102e-16,
         4.4409e-16,  4.4409e-16, -4.4409e-16, -8.8818e-16,  0.0000e+00,
         0.0000e+00,  0.0000e+00, -8.3267e-17,  0.0000e+00,  0.0000e+00,
        -8.8818e-16, -2.2204e-16,  1.1102e-16, -4.4409e-16,  3.0531e-16,
         1.1102e-16,  0.0000e+00,  2.2204e-16,  4.4409e-16,  6.6613e-16,
        -6.6613e-16,  0.0000e+00,  0.0000e+00, -8.8818e-16,  4.4409e-16,
        -4.4409e-16, -1.1102e-16,  2.2204e-16, -4.4409e-16,  0.0000e+00,
         0.0000e+00, -2.2204e-16,  0.0000e+00,  4.4

In [59]:
# %load ../mypkg/models/base_model.py
import torch
from hdf_utils.likelihood import obt_lin_tm


class BaseModel():
    """Base model
    """
    def __init__(self, Y, Z, X, basis_mat, ws=None):
        """
        args:
               Y: response values: M
               Z: matrix or vector of other covariates, (M) x q
               X: freq of data: (M) x d x npts
               basis_mat: Basis matrix of B-spline evaluated at some pts: npts x N
               ws: the weights used for approximating the integration: npts. 
        """
        if ws is None:
            ws = torch.ones(basis_mat.shape[0])/basis_mat.shape[0]
        self.ws = ws
        self.basis_mat = basis_mat
        self.X = X
        self.Z = Z
        self.Y = Y
    
    def _obt_lin_tm(self, alp, Gam):
        """Give the linear terms of likelihood fn
           args: 
               alp: parameters for Z: q
               Gam: parameters of B-spline: N x d
            return:
               lin_tm: the linear terms: scalar or vector of M
        """
        return obt_lin_tm(self.Z, self.X, alp, Gam, self.basis_mat, self.ws)
    
    def _linear_term_der(self):
        """
        # derivative of linear term w.r.t (alp, N^{-1/2}*Gam)
        It is a constant
        """
        basis_mat_trans = self.basis_mat.unsqueeze(0).unsqueeze(-1) # 1 x ncpts x N x 1
        X_trans = self.X.permute((0, 2, 1)).unsqueeze(2) # M x ncpts x 1 x d
        
        # derivative of linear term w.r.t (alp, N^{-1/2}*Gam)
        vec_part2_raw = basis_mat_trans*X_trans
        vec_part2_raw = vec_part2_raw.permute((0, 1, 3, 2)).flatten(2)
        vec_part2 = vec_part2_raw*self.ws.unsqueeze(0).unsqueeze(-1)
        vec_part2 = vec_part2.sum(axis=1)*np.sqrt(self.basis_mat.shape[1])
        lin_tm_der = torch.concat([self.Z, vec_part2], axis=1) #M x (q+dxN)
        return lin_tm_der

In [60]:
test = BaseModel(Y=Y, Z=Z, X=X, basis_mat=basis_mat)

In [62]:
test._linear_term_der().shape

torch.Size([100, 225])